# 匯入pkg

In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time, re
import copy

# Function 集合

## 2018/01到2020/04的唯一值音樂人的Function

In [2]:
def output(year, week, name_link, cat):
    """
    year = 年
    week = 週
    name_link = 音樂人的unique id
    cat = 音樂種類; 嘻哈/饒舌/雷鬼/Funk = 3, 舞曲/Remix/ACG/電音 = 4, R&B/靈魂/藍調/爵士/拉丁 = 5
    """
    for i in range(1, int(week)):
        dom = requests.get("https://streetvoice.com/music/charts/" + str(cat) + "/"  + str(year) + "/" + str(i) + "/").text #各種類網頁
        soup = BeautifulSoup(dom, 'html.parser')
        ele = soup.find_all("div", class_ = "work-item-info")
        
        try:
            for i in ele:
                nm_lk = i.find('h5').a['href']
                name_link.append(nm_lk)
        except Exception as e:
            print('')

## 抓取音樂人id

In [3]:
def findid(id_list):
    ele = soup.find_all("ul", class_ = "list-group list-group-flush list-group-song list-group-my-chart mb-3")

    for i in ele:
        identity = i.find('h5', 'text-truncate').a['href'].strip('/')
        print(identity)
        id_list.append(identity)
    return id_list

## 抓取音樂數、粉絲人數、追蹤人數的Function

In [4]:
def numfanfol(num, fans_num, follow_num):
    e = soup.find_all('h4', 'mb-0 font-size-h1') #抓取種類以及居住地的sector
    count = 1
    for i in e:
        if count == 1: #第一個為音樂
            info = i.text.strip()
            print(info)
            num.append(info)
            count += 1
        elif count == 2: #加1之後為粉絲
            info = i.text.strip()
            print(info)
            fans_num.append(info)
            count += 1
        else:
            info = i.text.strip()
            print(info)
            follow_num.append(info)
    return(num, fans_num, follow_num)

## 抓取種類以及居住地的Function

In [5]:
def typelive(tp, live):
    e = soup.find_all('span', 'mr-3') #抓取種類以及居住地的sector
    count = 1
    for i in e:
        if count == 1: #第一個為種類
            info = i.text.strip()
            print(info)
            tp.append(info)
            count += 1
        else: #加1之後為居住地
            info = i.text.strip()
            print(info)
            live.append(info)
            count += 1
    if count != 3:
        live.append('None')
        print('None')
    return(tp, live)

## 抓取Fans還有Follower's Link的Function

In [6]:
def fansfollowerlink(fans_link, follower_link):
    e = soup.find_all('li', 'list-inline-item ml-3')

    count = 1
    for i in e:
        if count == 1: #第一個為followers
            lk = str("https://streetvoice.com") + str(i.a['href']) #followers' link
            print(lk)
            fans_link.append(lk)
            count += 1
        else: #加1之後為following
            lk = str("https://streetvoice.com") + str(i.a['href']) #following link
            print(lk)
            follow_link.append(lk)
    return(fans_link, follow_link)

## 抓取Fans以及Follower's Unqiue ID 網頁爬蟲的Function

In [7]:
def fanfolwebcrawler(fanfollink, fanfol, fanfol_type, fanfol_num):
    url = fanfollink #連結fans以及follow的link
    driver = webdriver.Chrome(executable_path = 'chromedriver.exe')
    driver.implicitly_wait(10) #延遲讀網頁，使所有動態跑完
    #driver.maximize_window()
    driver.set_page_load_timeout(600) #最多等60秒
    driver.get(url)
    
    print(fanfol_num)
    
    try:
        element = driver.find_element_by_id('js-loader') #按下點我看更多按鈕
        element.click()
    except:
        pass

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
    soup = BeautifulSoup(driver.page_source, 'html5lib')
        
    driver.quit()
        
    e = soup.find_all('ul', 'list-group list-group-flush list-group-song list-group-onebtn mb-5') #抓取fans的種類
        
    for name in e:
        x = name.find_all()
        for i in x:
            a = i.text
        #temp.append(name.a['href'].strip('/'))
    
    temp1 = [] #暫時儲存fans的unique_id，等之後整個append到fans中
    temp2 = [] #暫時儲存fans的type，等之後整個append到fans_type中
    for elements in e:
        name = elements.find_all('h4', 'text-truncate')
        for ind in name:
            #print(ind.a['href'].strip('/'))
            temp1.append(ind.a['href'].strip('/'))
        ftype = elements.find_all('h5', 'text-truncate')
        for ind in ftype:
            #print(ind.text)
            temp2.append(ind.text)
    
    fanfol.append(temp1)
    fanfol_type.append(temp2)
    return(fanfol, fanfol_type)

## 抓取Fans以及Followe's 中有音樂人以及認證音樂人個數的Function

In [8]:
def fanfolmusician(type_list, musician_list):
    for i in range(0, len(type_list)):
        tempcount = 0
        for j in range(0, len(type_list[i])):
            if type_list[i][j] == '音樂人' or type_list[i][j] == '認證音樂人':
                tempcount += 1
        print(tempcount)
        musician_list.append(tempcount)
    return(musician_list)

# 表0-1: 嘻哈/饒舌/雷鬼/Funk

In [13]:
name_link = []
cat = 3

y1 = 2022; w1 = 20
y2 = 2021; w2 = 53
y3 = 2020; w3 = 54
y4 = 2019; w4 = 53
y5 = 2018; w5 = 53

output(y1, w1, name_link, cat)
output(y2, w2, name_link, cat)
output(y3, w3, name_link, cat)
output(y4, w4, name_link, cat)
output(y5, w5, name_link, cat)

name_link = list(set(name_link)) #取得唯一數值
df = pd.DataFrame(name_link)
export_csv = df.to_csv(r'嘻哈_饒舌_雷鬼_Funk_2.csv', header = False, index = False)

# 表0-2: 舞曲/Remix/ACG/電音

In [14]:
name_link = []
cat = 4

y1 = 2022; w1 = 20
y2 = 2021; w2 = 53
y3 = 2020; w3 = 54
y4 = 2019; w4 = 53
y5 = 2018; w5 = 53

output(y1, w1, name_link, cat)
output(y2, w2, name_link, cat)
output(y3, w3, name_link, cat)
output(y4, w4, name_link, cat)
output(y5, w5, name_link, cat)

name_link = list(set(name_link)) #取得唯一數值
df = pd.DataFrame(name_link)  
export_csv = df.to_csv(r'舞曲_Remix_ACG_電音_2.csv', header = False, index = False)

# 表0-3: R&B/靈魂/藍調/爵士/拉丁

In [15]:
name_link = []
cat = 5

y1 = 2022; w1 = 20
y2 = 2021; w2 = 53
y3 = 2020; w3 = 54
y4 = 2019; w4 = 53
y5 = 2018; w5 = 53

output(y1, w1, name_link, cat)
output(y2, w2, name_link, cat)
output(y3, w3, name_link, cat)
output(y4, w4, name_link, cat)
output(y5, w5, name_link, cat)

name_link = list(set(name_link)) #取得唯一數值
df = pd.DataFrame(name_link)
export_csv = df.to_csv(r'R&B_靈魂_藍調_爵士_拉丁_2.csv', header = False, index = False)

# 匯入各類型CSV檔案

## 嘻哈/饒舌/雷鬼/Funk

In [9]:
data_hip = pd.read_csv('嘻哈_饒舌_雷鬼_Funk_2.csv', header = None)
hip = data_hip.stack().tolist()

for i in range(0, len(hip)):
    hip[i] = str("https://streetvoice.com") + hip[i]
    
print(len(hip))
print(hip)

401
['https://streetvoice.com/BackMountKrew/', 'https://streetvoice.com/ape101129/', 'https://streetvoice.com/kaorecords/', 'https://streetvoice.com/TinTenTan/', 'https://streetvoice.com/hustlecrew330/', 'https://streetvoice.com/Lewis_chill/', 'https://streetvoice.com/ENWHY/', 'https://streetvoice.com/6XT7/', 'https://streetvoice.com/tsengsean/', 'https://streetvoice.com/depbread/', 'https://streetvoice.com/Vilen1222/', 'https://streetvoice.com/coujfever/', 'https://streetvoice.com/Chen_ni/', 'https://streetvoice.com/Hobrewblu/', 'https://streetvoice.com/softlipa/', 'https://streetvoice.com/vbnm1231/', 'https://streetvoice.com/somemusic/', 'https://streetvoice.com/redleey_3/', 'https://streetvoice.com/dcdessy/', 'https://streetvoice.com/gaojiafeng/', 'https://streetvoice.com/andung1220/', 'https://streetvoice.com/ShikinRollin/', 'https://streetvoice.com/peach_jaye_23/', 'https://streetvoice.com/csimusic/', 'https://streetvoice.com/Hupesoftman/', 'https://streetvoice.com/banyangang/', '

### 表1:抓音樂人的個別資料

In [10]:
hip_id = [] #音樂人的id
tp = [] #種類
live = [] #居住地
num = [] #音樂數量
fans_num = [] #粉絲數量
follow_num = [] #追蹤數量
fans_link = [] #粉絲網頁連結
follow_link = [] #追蹤網頁連結
fans = [] #粉絲
fans_type = [] #粉絲種類
follow = [] #追蹤
folow_type = [] #追蹤種類

In [14]:
for i in range(0, len(hip)):
    # 動態爬蟲、主要作用為延遲爬蟲時間
    url = hip[i]
    driver = webdriver.Chrome(executable_path = 'chromedriver.exe') #開啟虛擬器
    driver.maximize_window()
    driver.set_page_load_timeout(600) #最多等600秒
    driver.get(url)
    time.sleep(5) #延遲讀網頁，使所有動態跑完
    
    soup = BeautifulSoup(driver.page_source, 'html5lib')
    
    driver.quit() # 關閉虛擬器
    
    #抓取音樂人id
    findid(hip_id)
    
    # 抓取音樂數、粉絲人數、追蹤人數
    numfanfol(num, fans_num, follow_num)
    
    # 抓取種類以及居住地
    typelive(tp, live)
    
    # 抓取Fans還有Follower's Link的Function
    fansfollowerlink(fans_link, follow_link)
    print(i)
    print('---------------------------------------------')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


BackMountKrew
20
15,533
2
音樂人
花蓮縣
https://streetvoice.com/BackMountKrew/followers/
https://streetvoice.com/BackMountKrew/following/
0
---------------------------------------------
ape101129
14
207
56
認證音樂人
苗栗縣
https://streetvoice.com/ape101129/followers/
https://streetvoice.com/ape101129/following/
1
---------------------------------------------
kaorecords
21
3,332
16
音樂人
None
https://streetvoice.com/kaorecords/followers/
https://streetvoice.com/kaorecords/following/
2
---------------------------------------------
TinTenTan
14
17,318
12
音樂人
新北市
https://streetvoice.com/TinTenTan/followers/
https://streetvoice.com/TinTenTan/following/
3
---------------------------------------------
hustlecrew330
23
155
5
音樂人
桃園市
https://streetvoice.com/hustlecrew330/followers/
https://streetvoice.com/hustlecrew330/following/
4
---------------------------------------------
Lewis_chill
1
59
9
音樂人
新竹縣
https://streetvoice.com/Lewis_chill/followers/
https://streetvoice.com/Lewis_chill/following/
5
-----------

KEEEEEEXI
3
62
0
音樂人
廣東
https://streetvoice.com/KEEEEEEXI/followers/
https://streetvoice.com/KEEEEEEXI/following/
49
---------------------------------------------
johnnnnnn
22
1,791
49
認證音樂人
新北市
https://streetvoice.com/johnnnnnn/followers/
https://streetvoice.com/johnnnnnn/following/
50
---------------------------------------------
Bear_fan
12
224
36
認證音樂人
基隆市
https://streetvoice.com/Bear_fan/followers/
https://streetvoice.com/Bear_fan/following/
51
---------------------------------------------
longhumen
1
272
0
音樂人
基隆市
https://streetvoice.com/longhumen/followers/
https://streetvoice.com/longhumen/following/
52
---------------------------------------------
DUF
25
2,418
16
認證音樂人
新北市
https://streetvoice.com/DUF/followers/
https://streetvoice.com/DUF/following/
53
---------------------------------------------
Zoethechef
1
122
1
音樂人
臺中市
https://streetvoice.com/Zoethechef/followers/
https://streetvoice.com/Zoethechef/following/
54
---------------------------------------------
proaknpunnz
14

chinggsquadofficial
4
1,884
0
音樂人
臺北市
https://streetvoice.com/chinggsquadofficial/followers/
https://streetvoice.com/chinggsquadofficial/following/
98
---------------------------------------------
24prbl
19
575
19
認證音樂人
臺東縣
https://streetvoice.com/24prbl/followers/
https://streetvoice.com/24prbl/following/
99
---------------------------------------------
asd8983530
3
332
13
音樂人
新北市
https://streetvoice.com/asd8983530/followers/
https://streetvoice.com/asd8983530/following/
100
---------------------------------------------
FormoSir
3
83
2
音樂人
臺北市
https://streetvoice.com/FormoSir/followers/
https://streetvoice.com/FormoSir/following/
101
---------------------------------------------
NCCU_heyin
7
5,211
31
音樂人
臺北市
https://streetvoice.com/NCCU_heyin/followers/
https://streetvoice.com/NCCU_heyin/following/
102
---------------------------------------------
Rommyrollin
5
256
2
認證音樂人
屏東縣
https://streetvoice.com/Rommyrollin/followers/
https://streetvoice.com/Rommyrollin/following/
103
-----------

leothreeseven
15
1,358
0
音樂人
None
https://streetvoice.com/leothreeseven/followers/
https://streetvoice.com/leothreeseven/following/
146
---------------------------------------------
5400_54oo
2
729
0
認證音樂人
臺北市
https://streetvoice.com/5400_54oo/followers/
https://streetvoice.com/5400_54oo/following/
147
---------------------------------------------
Jinboismyname
25
1,068
4
認證音樂人
臺北市
https://streetvoice.com/Jinboismyname/followers/
https://streetvoice.com/Jinboismyname/following/
148
---------------------------------------------
zealots_45
1
307
0
音樂人
臺北市
https://streetvoice.com/zealots_45/followers/
https://streetvoice.com/zealots_45/following/
149
---------------------------------------------
The_Time
11
104
4
認證音樂人
台灣
https://streetvoice.com/The_Time/followers/
https://streetvoice.com/The_Time/following/
150
---------------------------------------------
charlie86847095
6
320
2
認證音樂人
新北市
https://streetvoice.com/charlie86847095/followers/
https://streetvoice.com/charlie86847095/followin

Chill_Bros
9
520
3
音樂人
臺北市
https://streetvoice.com/Chill_Bros/followers/
https://streetvoice.com/Chill_Bros/following/
194
---------------------------------------------
dusanaga
35
563
0
音樂人
新北市
https://streetvoice.com/dusanaga/followers/
https://streetvoice.com/dusanaga/following/
195
---------------------------------------------
wring_out_laura
40
188
28
認證音樂人
臺北市
https://streetvoice.com/wring_out_laura/followers/
https://streetvoice.com/wring_out_laura/following/
196
---------------------------------------------
dinlow
28
408
135
認證音樂人
臺北市
https://streetvoice.com/dinlow/followers/
https://streetvoice.com/dinlow/following/
197
---------------------------------------------
bambii_annz
9
240
5
認證音樂人
臺北市
https://streetvoice.com/bambii_annz/followers/
https://streetvoice.com/bambii_annz/following/
198
---------------------------------------------
rebecca1015
2
156
7
音樂人
臺北市
https://streetvoice.com/rebecca1015/followers/
https://streetvoice.com/rebecca1015/following/
199
-----------------

dsa265302
2
44
6
音樂人
苗栗縣
https://streetvoice.com/dsa265302/followers/
https://streetvoice.com/dsa265302/following/
242
---------------------------------------------
suckmc
4
3,352
1
音樂人
台灣
https://streetvoice.com/suckmc/followers/
https://streetvoice.com/suckmc/following/
243
---------------------------------------------
catwei
9
188
1
音樂人
臺南市
https://streetvoice.com/catwei/followers/
https://streetvoice.com/catwei/following/
244
---------------------------------------------
shuya_
3
8,553
2
音樂人
新北市
https://streetvoice.com/shuya_/followers/
https://streetvoice.com/shuya_/following/
245
---------------------------------------------
KayC14
4
397
3
認證音樂人
新北市
https://streetvoice.com/KayC14/followers/
https://streetvoice.com/KayC14/following/
246
---------------------------------------------
Piper_peng
9
153
0
音樂人
嘉義市
https://streetvoice.com/Piper_peng/followers/
https://streetvoice.com/Piper_peng/following/
247
---------------------------------------------
ozioffcl
11
18,059
2
音樂人
臺北市
http

OFF_THE_RECORD_official
35
114
0
音樂人
臺北市
https://streetvoice.com/OFF_THE_RECORD_official/followers/
https://streetvoice.com/OFF_THE_RECORD_official/following/
291
---------------------------------------------
Rooftopmob
39
384
8
認證音樂人
新北市
https://streetvoice.com/Rooftopmob/followers/
https://streetvoice.com/Rooftopmob/following/
292
---------------------------------------------
tzuyuan30
13
122
0
認證音樂人
臺南市
https://streetvoice.com/tzuyuan30/followers/
https://streetvoice.com/tzuyuan30/following/
293
---------------------------------------------
goldink0515
6
375
6
認證音樂人
臺北市
https://streetvoice.com/goldink0515/followers/
https://streetvoice.com/goldink0515/following/
294
---------------------------------------------
ComradeXiangyu
15
292
7
音樂人
新北市
https://streetvoice.com/ComradeXiangyu/followers/
https://streetvoice.com/ComradeXiangyu/following/
295
---------------------------------------------
dropp
40
238
15
認證音樂人
美國
https://streetvoice.com/dropp/followers/
https://streetvoice.com/drop

catsonslide2019
15
395
8
音樂人
新北市
https://streetvoice.com/catsonslide2019/followers/
https://streetvoice.com/catsonslide2019/following/
339
---------------------------------------------
rosehuang1127
4
148
17
認證音樂人
臺北市
https://streetvoice.com/rosehuang1127/followers/
https://streetvoice.com/rosehuang1127/following/
340
---------------------------------------------
babybabycrycry
3
390
19
認證音樂人
臺南市
https://streetvoice.com/babybabycrycry/followers/
https://streetvoice.com/babybabycrycry/following/
341
---------------------------------------------
still_king_q5
2
15
26
音樂人
臺中市
https://streetvoice.com/still_king_q5/followers/
https://streetvoice.com/still_king_q5/following/
342
---------------------------------------------
kkee12345
5
38
28
音樂人
新北市
https://streetvoice.com/kkee12345/followers/
https://streetvoice.com/kkee12345/following/
343
---------------------------------------------
happy2031451
25
769
28
認證音樂人
臺北市
https://streetvoice.com/happy2031451/followers/
https://streetvoice.com/h

3r2music
9
359
4
音樂人
臺北市
https://streetvoice.com/3r2music/followers/
https://streetvoice.com/3r2music/following/
387
---------------------------------------------
fourpens
28
4,266
63
認證音樂人
臺北市
https://streetvoice.com/fourpens/followers/
https://streetvoice.com/fourpens/following/
388
---------------------------------------------
hitdondon
16
2,047
0
認證音樂人
高雄市
https://streetvoice.com/hitdondon/followers/
https://streetvoice.com/hitdondon/following/
389
---------------------------------------------
s9021720000
1
33
20
音樂人
桃園市
https://streetvoice.com/s9021720000/followers/
https://streetvoice.com/s9021720000/following/
390
---------------------------------------------
c8heihei
2
35
10
音樂人
臺南市
https://streetvoice.com/c8heihei/followers/
https://streetvoice.com/c8heihei/following/
391
---------------------------------------------
Not_Yet_1
2
234
2
音樂人
桃園市
https://streetvoice.com/Not_Yet_1/followers/
https://streetvoice.com/Not_Yet_1/following/
392
------------------------------------------

In [15]:
print('hip_id:', hip_id)
print(len(hip_id))
print('num:', num)
print(len(num))
print('fans_num:', fans_num)
print(len(fans_num))
print('follow_num:', follow_num)
print(len(follow_num))
print('tp:', tp)
print(len(tp))
print('live:', live)
print(len(live))
print('fans_link:', fans_link)
print(len(fans_link))
print('follow_link:', follow_link)
print(len(follow_link))

hip_id: ['BackMountKrew', 'BackMountKrew', 'ape101129', 'kaorecords', 'TinTenTan', 'hustlecrew330', 'Lewis_chill', 'ENWHY', '6XT7', 'tsengsean', 'depbread', 'Vilen1222', 'coujfever', 'Chen_ni', 'Hobrewblu', 'softlipa', 'vbnm1231', 'somemusic', 'redleey_3', 'dcdessy', 'gaojiafeng', 'andung1220', 'ShikinRollin', 'peach_jaye_23', 'csimusic', 'Hupesoftman', 'banyangang', 'Charleschang', 'sync_with_me', 'look850802', 'chord76415', 'Chih1012', 'aquariusboys', 'Chenmax0', 'Shouttttttt', 'patrickjohnlu', 'wesley753951', 'linyuchang', 'yung_chiao', 'elimlee', 'LoadingStory_Official', 'aka_lulu', 'kailun0409', 'ByronHsu', 'zac60422', 'nabeyaki_egg_noodles', 'stone1024', 'tyntseng', 'BaconSlap', 'CuifengWizard54', 'KEEEEEEXI', 'johnnnnnn', 'Bear_fan', 'longhumen', 'DUF', 'Zoethechef', 'proaknpunnz', 'Chancharlie_', 'Fourmula_band', 'gabuiui', 'nooovaaa', 'noisy_haze_dd', 'DJGround', 'ourshame92', 'yu0965160616', 'ill_mo', 'mcdouble8', 'ZEO_WATER', 'ryota_katayama', 'wc109101065', 'SouthVibe', 'wo

### 抓取粉絲人數中，音樂人的人數

In [ ]:
fans = [] #粉絲
fans_type = [] #粉絲種類
fans_musician = [] #粉絲中有音樂人的數量
follow = [] #追蹤
follow_type = [] #追蹤種類
follow_musician = [] #追蹤中有音樂人的數量

In [ ]:
for i in range(0, len(fans_link)):
    fanfolwebcrawler(fans_link[i], fans, fans_type, fans_num[i]) #抓取粉絲名稱以及粉絲種類

In [ ]:
fanfolmusician(fans_type, fans_musician) #計算粉絲中音樂人以及認證音樂人的數量

### 追蹤對象中，音樂人的人數

In [ ]:
for i in range(0, len(follow_link)):
    fanfolwebcrawler(follow_link[i], follow, follow_type, follow_num[i]) #抓取粉絲名稱以及粉絲種類

In [ ]:
fanfolmusician(follow_type, follow_musician)

In [ ]:
print(fans)
print(len(fans))
print(fans_type)
print(len(fans_type))
print(fans_musician)
print(len(fans_musician))
print(follow)
print(len(follow))
print(follow_type)
print(len(follow_type))
print(follow_musician)
print(len(follow_musician))

In [ ]:
df1_1 = pd.DataFrame(zip(hip_id, num, fans_num, follow_num, tp, live, fans_musician, follow_musician), 
                   columns = ["音樂人 ID", "音樂數", "粉絲人數", "追蹤中人數", "身分(音樂人/認證音樂人)",
                              "居住地", "粉絲人數中，音樂人的人數", "追蹤對象中，音樂人的人數"])
 
export_csv = df1_1.to_csv(r'df1_1.csv', header = True, index = False, encoding = "UTF-8")
print(df1_1)

In [16]:
df1_1 = pd.DataFrame(zip(hip_id, num, fans_num, follow_num, tp, live), 
                   columns = ["音樂人 ID", "音樂數", "粉絲人數", "追蹤中人數", "身分(音樂人/認證音樂人)",
                              "居住地"])
 
export_csv = df1_1.to_csv(r'df1_1_2.csv', header = True, index = False, encoding = "UTF-8")
print(df1_1)

              音樂人 ID 音樂數    粉絲人數 追蹤中人數 身分(音樂人/認證音樂人)   居住地
0      BackMountKrew  20  15,533     2           音樂人   花蓮縣
1      BackMountKrew  20  15,533     2           音樂人   花蓮縣
2          ape101129  14     207    56         認證音樂人   苗栗縣
3         kaorecords  21   3,332    16           音樂人  None
4          TinTenTan  14  17,318    12           音樂人   新北市
..               ...  ..     ...   ...           ...   ...
397  tigtheirongiant  11   2,394    15         認證音樂人   臺北市
398         Acedalan   3      86    12           音樂人   新竹縣
399         flexkidz  19     124     3           音樂人   臺北市
400        blaire_ko  72     371     6           音樂人   臺北市
401      970zhaozhao   3     165    13         認證音樂人   臺北市

[402 rows x 6 columns]


## 舞曲_Remix_ACG_電音

In [17]:
data_dance = pd.read_csv('舞曲_Remix_ACG_電音_2.csv', header = None)
dance = data_dance.stack().tolist()

for i in range(0, len(dance)):
    dance[i] = str("https://streetvoice.com") + dance[i]
    
print(len(dance))
print(dance)

256
['https://streetvoice.com/itisIsey/', 'https://streetvoice.com/Darkduckisme/', 'https://streetvoice.com/chenx3/', 'https://streetvoice.com/koala_dada/', 'https://streetvoice.com/J_wang_29/', 'https://streetvoice.com/ovdstw/', 'https://streetvoice.com/oscar99918/', 'https://streetvoice.com/wavesdoppler/', 'https://streetvoice.com/ape101129/', 'https://streetvoice.com/ella26980403/', 'https://streetvoice.com/chenghantw/', 'https://streetvoice.com/GoGoMachineOrchy/', 'https://streetvoice.com/b0002381/', 'https://streetvoice.com/dungisapor/', 'https://streetvoice.com/hilloper/', 'https://streetvoice.com/EastWaveBand/', 'https://streetvoice.com/pilot/', 'https://streetvoice.com/Sonia_Calico/', 'https://streetvoice.com/neonerostw/', 'https://streetvoice.com/som4ever/', 'https://streetvoice.com/Locu5/', 'https://streetvoice.com/AOIMUSHIBAND/', 'https://streetvoice.com/nimoyully/', 'https://streetvoice.com/AngelHeartBand/', 'https://streetvoice.com/Iron_Punch_band/', 'https://streetvoice.c

In [18]:
dance_id = [] #音樂人的id
tp = [] #種類
live = [] #居住地
num = [] #音樂數量
fans_num = [] #粉絲數量
follow_num = [] #追蹤數量
fans_link = [] #粉絲網頁連結
follow_link = [] #追蹤網頁連結
fans = [] #粉絲
fans_type = [] #粉絲種類
follow = [] #追蹤
folow_type = [] #追蹤種類

In [19]:
for i in range(0, len(dance)):
    # 動態爬蟲、主要作用為延遲爬蟲時間
    url = dance[i]
    driver = webdriver.Chrome(executable_path = 'chromedriver.exe') #開啟虛擬器
    driver.maximize_window()
    driver.set_page_load_timeout(600) #最多等600秒
    driver.get(url)
    time.sleep(5) #延遲讀網頁，使所有動態跑完
    
    soup = BeautifulSoup(driver.page_source, 'html5lib')
    
    driver.quit() # 關閉虛擬器
    
    #抓取音樂人id
    findid(dance_id)
    
    # 抓取音樂數、粉絲人數、追蹤人數
    numfanfol(num, fans_num, follow_num)
    
    # 抓取種類以及居住地
    typelive(tp, live)
    
    # 抓取Fans還有Follower's Link的Function
    fansfollowerlink(fans_link, follow_link)
    print(i)
    print('---------------------------------------------')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


itisIsey
22
751
14
認證音樂人
臺北市
https://streetvoice.com/itisIsey/followers/
https://streetvoice.com/itisIsey/following/
0
---------------------------------------------
Darkduckisme
1
15
34
音樂人
臺中市
https://streetvoice.com/Darkduckisme/followers/
https://streetvoice.com/Darkduckisme/following/
1
---------------------------------------------
chenx3
16
160
0
音樂人
浙江
https://streetvoice.com/chenx3/followers/
https://streetvoice.com/chenx3/following/
2
---------------------------------------------
koala_dada
5
10
39
音樂人
新北市
https://streetvoice.com/koala_dada/followers/
https://streetvoice.com/koala_dada/following/
3
---------------------------------------------
J_wang_29
1
24
3
音樂人
桃園市
https://streetvoice.com/J_wang_29/followers/
https://streetvoice.com/J_wang_29/following/
4
---------------------------------------------
ovdstw
33
926
37
認證音樂人
臺北市
https://streetvoice.com/ovdstw/followers/
https://streetvoice.com/ovdstw/following/
5
---------------------------------------------
oscar99918
1
4
0
音

kalabebe
4
373
118
認證音樂人
新北市
https://streetvoice.com/kalabebe/followers/
https://streetvoice.com/kalabebe/following/
49
---------------------------------------------
roctordoctor
1
28
12
音樂人
臺北市
https://streetvoice.com/roctordoctor/followers/
https://streetvoice.com/roctordoctor/following/
50
---------------------------------------------
hellovivianana
4
13
1
音樂人
臺北市
https://streetvoice.com/hellovivianana/followers/
https://streetvoice.com/hellovivianana/following/
51
---------------------------------------------
TK_supremacy
5
141
5
認證音樂人
臺南市
https://streetvoice.com/TK_supremacy/followers/
https://streetvoice.com/TK_supremacy/following/
52
---------------------------------------------
DSPS
20
11,926
6
認證音樂人
臺北市
https://streetvoice.com/DSPS/followers/
https://streetvoice.com/DSPS/following/
53
---------------------------------------------
gaojiafeng
16
1,083
4
音樂人
上海
https://streetvoice.com/gaojiafeng/followers/
https://streetvoice.com/gaojiafeng/following/
54
-------------------------

sparkylee
29
169
12
音樂人
歐洲
https://streetvoice.com/sparkylee/followers/
https://streetvoice.com/sparkylee/following/
98
---------------------------------------------
ruby200457630
8
108
12
認證音樂人
新竹市
https://streetvoice.com/ruby200457630/followers/
https://streetvoice.com/ruby200457630/following/
99
---------------------------------------------
kuroshio2019
9
1,055
0
認證音樂人
臺北市
https://streetvoice.com/kuroshio2019/followers/
https://streetvoice.com/kuroshio2019/following/
100
---------------------------------------------
kokkamango
42
81
36
音樂人
馬來西亞
https://streetvoice.com/kokkamango/followers/
https://streetvoice.com/kokkamango/following/
101
---------------------------------------------
dAAAb
1
10
0
音樂人
臺北市
https://streetvoice.com/dAAAb/followers/
https://streetvoice.com/dAAAb/following/
102
---------------------------------------------
zac60422
58
966
81
認證音樂人
新北市
https://streetvoice.com/zac60422/followers/
https://streetvoice.com/zac60422/following/
103
------------------------------

sodoesthedog
14
843
19
認證音樂人
新北市
https://streetvoice.com/sodoesthedog/followers/
https://streetvoice.com/sodoesthedog/following/
147
---------------------------------------------
faye_official
24
2,402
5
認證音樂人
臺北市
https://streetvoice.com/faye_official/followers/
https://streetvoice.com/faye_official/following/
148
---------------------------------------------
tobemusictw
11
134
0
認證音樂人
臺北市
https://streetvoice.com/tobemusictw/followers/
https://streetvoice.com/tobemusictw/following/
149
---------------------------------------------
finn0829
14
144
45
認證音樂人
臺北市
https://streetvoice.com/finn0829/followers/
https://streetvoice.com/finn0829/following/
150
---------------------------------------------
yutinhuang
9
198
23
認證音樂人
臺北市
https://streetvoice.com/yutinhuang/followers/
https://streetvoice.com/yutinhuang/following/
151
---------------------------------------------
michaellee1995
7
28
8
音樂人
桃園市
https://streetvoice.com/michaellee1995/followers/
https://streetvoice.com/michaellee1995/follo

KOzZY
7
47
16
音樂人
臺中市
https://streetvoice.com/KOzZY/followers/
https://streetvoice.com/KOzZY/following/
196
---------------------------------------------
sweetsleep
16
428
1
認證音樂人
臺北市
https://streetvoice.com/sweetsleep/followers/
https://streetvoice.com/sweetsleep/following/
197
---------------------------------------------
hsiangyukao
19
358
107
認證音樂人
臺北市
https://streetvoice.com/hsiangyukao/followers/
https://streetvoice.com/hsiangyukao/following/
198
---------------------------------------------
cream_music
1
1
0
音樂人
臺中市
https://streetvoice.com/cream_music/followers/
https://streetvoice.com/cream_music/following/
199
---------------------------------------------
crybabymush
5
5
1
音樂人
高雄市
https://streetvoice.com/crybabymush/followers/
https://streetvoice.com/crybabymush/following/
200
---------------------------------------------
dapunmusic
9
45
1
音樂人
臺北市
https://streetvoice.com/dapunmusic/followers/
https://streetvoice.com/dapunmusic/following/
201
-----------------------------------

babelrecords
60
636
5
音樂人
北京
https://streetvoice.com/babelrecords/followers/
https://streetvoice.com/babelrecords/following/
244
---------------------------------------------
BeatmakersTaipei
59
1,794
0
認證音樂人
臺北市
https://streetvoice.com/BeatmakersTaipei/followers/
https://streetvoice.com/BeatmakersTaipei/following/
245
---------------------------------------------
L8ching
5
1,758
0
認證音樂人
臺北市
https://streetvoice.com/L8ching/followers/
https://streetvoice.com/L8ching/following/
246
---------------------------------------------
shawn_kao
6
19
14
音樂人
新北市
https://streetvoice.com/shawn_kao/followers/
https://streetvoice.com/shawn_kao/following/
247
---------------------------------------------
vois
2
11
1
音樂人
臺中市
https://streetvoice.com/vois/followers/
https://streetvoice.com/vois/following/
248
---------------------------------------------
turtleknights
4
1,713
5
音樂人
None
https://streetvoice.com/turtleknights/followers/
https://streetvoice.com/turtleknights/following/
249
------------------

In [20]:
print('dance_id:', dance_id)
print(len(dance_id))
print('num:', num)
print(len(num))
print('fans_num:', fans_num)
print(len(fans_num))
print('follow_num:', follow_num)
print(len(follow_num))
print('tp:', tp)
print(len(tp))
print('live:', live)
print(len(live))
print('fans_link:', fans_link)
print(len(fans_link))
print('follow_link:', follow_link)
print(len(follow_link))

dance_id: ['itisIsey', 'Darkduckisme', 'chenx3', 'koala_dada', 'J_wang_29', 'ovdstw', 'oscar99918', 'wavesdoppler', 'ape101129', 'ella26980403', 'chenghantw', 'GoGoMachineOrchy', 'b0002381', 'dungisapor', 'hilloper', 'EastWaveBand', 'pilot', 'Sonia_Calico', 'neonerostw', 'som4ever', 'Locu5', 'AOIMUSHIBAND', 'nimoyully', 'AngelHeartBand', 'Iron_Punch_band', 'abc898277', 'Onstudio', 'MidisWonderland', 'vegatablecookies', 'chickenchicks', 'troychenGT', 'ZooeyWonder', 'PuFFcorn', 'theloophole2017', 'OldSlutDistortion', 'chilladan', 'runningyouth', 'softlipa', 'BURNIE', 'rubyfatale', 'WithinTheFlames', 'MLK1992', 'Oberka', 'hanahsu', 'mibsBMty330', 'gac900423', 'MrLH_Band', 'Qzzy', 'ufa', 'kalabebe', 'roctordoctor', 'hellovivianana', 'TK_supremacy', 'DSPS', 'gaojiafeng', 'kai_xuan_159', 'AmberKuo_AK', 'TransferStudent', 'CHIUPI', 's_p_i_c_e', 'spaceoddity2021', 'Leepa', 'yes3861900', 'SYNTHGUMI04', 'owfband', 'solomomliu', 'rnbconey', 'zhcband', 'floatrist', 'city_island', 'hsinchen0002', '

In [ ]:
fans = [] #粉絲
fans_type = [] #粉絲種類
fans_musician = [] #粉絲中有音樂人的數量
follow = [] #追蹤
follow_type = [] #追蹤種類
follow_musician = [] #追蹤中有音樂人的數量

In [ ]:
for i in range(0, len(fans_link)):
    fanfolwebcrawler(fans_link[i], fans, fans_type, fans_num[i]) #抓取粉絲名稱以及粉絲種類

In [ ]:
fanfolmusician(fans_type, fans_musician) #計算粉絲中音樂人以及認證音樂人的數量

In [ ]:
for i in range(0, len(follow_link)):
    fanfolwebcrawler(follow_link[i], follow, follow_type, follow_num[i]) #抓取粉絲名稱以及粉絲種類

In [ ]:
fanfolmusician(follow_type, follow_musician)

In [ ]:
print(fans)
print(len(fans))
print(fans_type)
print(len(fans_type))
print(fans_musician)
print(len(fans_musician))
print(follow)
print(len(follow))
print(follow_type)
print(len(follow_type))
print(follow_musician)
print(len(follow_musician))

In [ ]:
df1_2 = pd.DataFrame(zip(dance_id, num, fans_num, follow_num, tp, live, fans_musician, follow_musician), 
                   columns = ["音樂人 ID", "音樂數", "粉絲人數", "追蹤中人數", "身分(音樂人/認證音樂人)",
                              "居住地", "粉絲人數中，音樂人的人數", "追蹤對象中，音樂人的人數"])
 
export_csv = df1_2.to_csv(r'df1_2.csv', header = True, index = False, encoding = "UTF-8")
print(df1_2)

In [21]:
df1_2 = pd.DataFrame(zip(dance_id, num, fans_num, follow_num, tp, live), 
                   columns = ["音樂人 ID", "音樂數", "粉絲人數", "追蹤中人數", "身分(音樂人/認證音樂人)",
                              "居住地"])
 
export_csv = df1_2.to_csv(r'df1_2_2.csv', header = True, index = False, encoding = "UTF-8")
print(df1_2)

                音樂人 ID 音樂數   粉絲人數 追蹤中人數 身分(音樂人/認證音樂人)  居住地
0             itisIsey  22    751    14         認證音樂人  臺北市
1         Darkduckisme   1     15    34           音樂人  臺中市
2               chenx3  16    160     0           音樂人   浙江
3           koala_dada   5     10    39           音樂人  新北市
4            J_wang_29   1     24     3           音樂人  桃園市
..                 ...  ..    ...   ...           ...  ...
251          AnnieTaco  34    585    24           音樂人  臺中市
252         fever86520   5  1,027    25         認證音樂人  新北市
253  madeinbackstation   5  5,246     6         認證音樂人  桃園市
254         tarzan0207   2     64     4           音樂人  臺北市
255          cheir_yun   8      9     0           音樂人  臺北市

[256 rows x 6 columns]


## R&B_靈魂_藍調_爵士_拉丁

In [22]:
data_rb = pd.read_csv('R&B_靈魂_藍調_爵士_拉丁_2.csv', header = None)
rb = data_rb.stack().tolist()

for i in range(0, len(rb)):
    rb[i] = str("https://streetvoice.com") + rb[i]
    
print(len(rb))
print(rb)

274
['https://streetvoice.com/buddhacomfortable/', 'https://streetvoice.com/sasa0720h/', 'https://streetvoice.com/ChuNoodleChuNoodle/', 'https://streetvoice.com/8bkuo/', 'https://streetvoice.com/anthony0926handsome/', 'https://streetvoice.com/ovdstw/', 'https://streetvoice.com/24prbl/', 'https://streetvoice.com/d1205x/', 'https://streetvoice.com/Wisely_d_d/', 'https://streetvoice.com/ohmymeiting/', 'https://streetvoice.com/zizimusic/', 'https://streetvoice.com/thegirlsony/', 'https://streetvoice.com/Ke2n1/', 'https://streetvoice.com/ISQ4/', 'https://streetvoice.com/tim99640996/', 'https://streetvoice.com/Fe_N/', 'https://streetvoice.com/jamesli123/', 'https://streetvoice.com/lil_du/', 'https://streetvoice.com/dachen223/', 'https://streetvoice.com/masterbating6969/', 'https://streetvoice.com/88risingRecordsLLC/', 'https://streetvoice.com/Iron_Punch_band/', 'https://streetvoice.com/cbn6rbp/', 'https://streetvoice.com/lekimberleyy/', 'https://streetvoice.com/_6vincent9_/', 'https://street

In [23]:
rb_id = [] #音樂人的id
tp = [] #種類
live = [] #居住地
num = [] #音樂數量
fans_num = [] #粉絲數量
follow_num = [] #追蹤數量
fans_link = [] #粉絲網頁連結
follow_link = [] #追蹤網頁連結
fans = [] #粉絲
fans_type = [] #粉絲種類
follow = [] #追蹤
folow_type = [] #追蹤種類

In [24]:
for i in range(0, len(rb)):
    # 動態爬蟲、主要作用為延遲爬蟲時間
    url = rb[i]
    driver = webdriver.Chrome(executable_path = 'chromedriver.exe') #開啟虛擬器
    driver.maximize_window()
    driver.set_page_load_timeout(600) #最多等600秒
    driver.get(url)
    time.sleep(5) #延遲讀網頁，使所有動態跑完
    
    soup = BeautifulSoup(driver.page_source, 'html5lib')
    
    driver.quit() # 關閉虛擬器
    
    #抓取音樂人id
    findid(rb_id)
    
    # 抓取音樂數、粉絲人數、追蹤人數
    numfanfol(num, fans_num, follow_num)
    
    # 抓取種類以及居住地
    typelive(tp, live)
    
    # 抓取Fans還有Follower's Link的Function
    fansfollowerlink(fans_link, follow_link)
    print(i)
    print('---------------------------------------------')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


buddhacomfortable
10
23
0
音樂人
臺中市
https://streetvoice.com/buddhacomfortable/followers/
https://streetvoice.com/buddhacomfortable/following/
0
---------------------------------------------
sasa0720h
11
1,649
12
認證音樂人
桃園市
https://streetvoice.com/sasa0720h/followers/
https://streetvoice.com/sasa0720h/following/
1
---------------------------------------------
ChuNoodleChuNoodle
11
741
46
認證音樂人
新北市
https://streetvoice.com/ChuNoodleChuNoodle/followers/
https://streetvoice.com/ChuNoodleChuNoodle/following/
2
---------------------------------------------
8bkuo
1
156
8
音樂人
新北市
https://streetvoice.com/8bkuo/followers/
https://streetvoice.com/8bkuo/following/
3
---------------------------------------------
anthony0926handsome
21
4,746
3
認證音樂人
臺北市
https://streetvoice.com/anthony0926handsome/followers/
https://streetvoice.com/anthony0926handsome/following/
4
---------------------------------------------
ovdstw
33
926
37
認證音樂人
臺北市
https://streetvoice.com/ovdstw/followers/
https://streetvoice.com/ovd

Conehead
51
734
157
音樂人
基隆市
https://streetvoice.com/Conehead/followers/
https://streetvoice.com/Conehead/following/
49
---------------------------------------------
aretall
1
274
3
認證音樂人
高雄市
https://streetvoice.com/aretall/followers/
https://streetvoice.com/aretall/following/
50
---------------------------------------------
genneo
17
529
1
音樂人
新加坡
https://streetvoice.com/genneo/followers/
https://streetvoice.com/genneo/following/
51
---------------------------------------------
dannyhungssss
4
40
9
音樂人
臺北市
https://streetvoice.com/dannyhungssss/followers/
https://streetvoice.com/dannyhungssss/following/
52
---------------------------------------------
Haormusic
93
1,039
15
音樂人
None
https://streetvoice.com/Haormusic/followers/
https://streetvoice.com/Haormusic/following/
53
---------------------------------------------
somemusic
4
78
0
音樂人
None
https://streetvoice.com/somemusic/followers/
https://streetvoice.com/somemusic/following/
54
---------------------------------------------
wayneb

catsonslide2019
15
395
8
音樂人
新北市
https://streetvoice.com/catsonslide2019/followers/
https://streetvoice.com/catsonslide2019/following/
97
---------------------------------------------
Caramelsauce_
2
24
2
音樂人
高雄市
https://streetvoice.com/Caramelsauce_/followers/
https://streetvoice.com/Caramelsauce_/following/
98
---------------------------------------------
Fomchor97
36
45
1
音樂人
上海
https://streetvoice.com/Fomchor97/followers/
https://streetvoice.com/Fomchor97/following/
99
---------------------------------------------
goldfishbrain
5
226
1
音樂人
臺北市
https://streetvoice.com/goldfishbrain/followers/
https://streetvoice.com/goldfishbrain/following/
100
---------------------------------------------
zcrighthere
29
1,199
15
音樂人
上海
https://streetvoice.com/zcrighthere/followers/
https://streetvoice.com/zcrighthere/following/
101
---------------------------------------------
Soulfomusic
5
25
1
音樂人
臺北市
https://streetvoice.com/Soulfomusic/followers/
https://streetvoice.com/Soulfomusic/following/
10

Jerry0426
5
33
1
音樂人
基隆市
https://streetvoice.com/Jerry0426/followers/
https://streetvoice.com/Jerry0426/following/
145
---------------------------------------------
dutchitrap
6
52
0
音樂人
臺北市
https://streetvoice.com/dutchitrap/followers/
https://streetvoice.com/dutchitrap/following/
146
---------------------------------------------
OvertoneTW
36
633
282
認證音樂人
臺南市
https://streetvoice.com/OvertoneTW/followers/
https://streetvoice.com/OvertoneTW/following/
147
---------------------------------------------
welchenmusic
46
32
0
音樂人
臺北市
https://streetvoice.com/welchenmusic/followers/
https://streetvoice.com/welchenmusic/following/
148
---------------------------------------------
Honwei
1
37
4
音樂人
南投縣
https://streetvoice.com/Honwei/followers/
https://streetvoice.com/Honwei/following/
149
---------------------------------------------
Fourmula_band
10
380
0
認證音樂人
臺北市
https://streetvoice.com/Fourmula_band/followers/
https://streetvoice.com/Fourmula_band/following/
150
---------------------------

franklintelescope
13
118
0
音樂人
香港
https://streetvoice.com/franklintelescope/followers/
https://streetvoice.com/franklintelescope/following/
194
---------------------------------------------
Noborusan
17
120
22
音樂人
臺北市
https://streetvoice.com/Noborusan/followers/
https://streetvoice.com/Noborusan/following/
195
---------------------------------------------
earthdayoff
8
545
6
音樂人
臺北市
https://streetvoice.com/earthdayoff/followers/
https://streetvoice.com/earthdayoff/following/
196
---------------------------------------------
TommyBlue
8
217
5
音樂人
臺北市
https://streetvoice.com/TommyBlue/followers/
https://streetvoice.com/TommyBlue/following/
197
---------------------------------------------
Yoannasun
3
429
0
音樂人
其他
https://streetvoice.com/Yoannasun/followers/
https://streetvoice.com/Yoannasun/following/
198
---------------------------------------------
mxxdlife
16
2,969
0
認證音樂人
新北市
https://streetvoice.com/mxxdlife/followers/
https://streetvoice.com/mxxdlife/following/
199
-----------------

sherryccexon
22
320
15
認證音樂人
臺北市
https://streetvoice.com/sherryccexon/followers/
https://streetvoice.com/sherryccexon/following/
243
---------------------------------------------
sean842850
6
5,139
17
認證音樂人
臺北市
https://streetvoice.com/sean842850/followers/
https://streetvoice.com/sean842850/following/
244
---------------------------------------------
applecat0228
3
43
0
音樂人
臺北市
https://streetvoice.com/applecat0228/followers/
https://streetvoice.com/applecat0228/following/
245
---------------------------------------------
Kminddddd
2
169
4
音樂人
香港
https://streetvoice.com/Kminddddd/followers/
https://streetvoice.com/Kminddddd/following/
246
---------------------------------------------
zaq1234_12367
5
141
30
音樂人
高雄市
https://streetvoice.com/zaq1234_12367/followers/
https://streetvoice.com/zaq1234_12367/following/
247
---------------------------------------------
wayne__dong
3
185
1
認證音樂人
桃園市
https://streetvoice.com/wayne__dong/followers/
https://streetvoice.com/wayne__dong/following/
248
-

In [25]:
print('rb_id:', rb_id)
print(len(rb_id))
print('num:', num)
print(len(num))
print('fans_num:', fans_num)
print(len(fans_num))
print('follow_num:', follow_num)
print(len(follow_num))
print('tp:', tp)
print(len(tp))
print('live:', live)
print(len(live))
print('fans_link:', fans_link)
print(len(fans_link))
print('follow_link:', follow_link)
print(len(follow_link))

rb_id: ['buddhacomfortable', 'sasa0720h', 'ChuNoodleChuNoodle', '8bkuo', 'anthony0926handsome', 'ovdstw', '24prbl', 'd1205x', 'Wisely_d_d', 'ohmymeiting', 'zizimusic', 'thegirlsony', 'Ke2n1', 'ISQ4', 'tim99640996', 'Fe_N', 'jamesli123', 'lil_du', 'dachen223', 'masterbating6969', '88risingRecordsLLC', 'Iron_Punch_band', 'cbn6rbp', 'lekimberleyy', '_6vincent9_', 'Onstudio', 'toufahutong', 'MidisWonderland', 'chickenchicks', 'lingyuankeng', 'yentinglo', 'PuFFcorn', 'Muniyu', 'chilladan', 'tainaner', 'David_0912', 'xiahoxiaho', 'nauhuang', 'JimmyStan', 'chenxen', 'Kevinchen408', 'rubyfatale', 'ArthurShun2018', 'Mo1028', 'mibsBMty330', 'felixanshen', 'ET_Tseng', 'angry_youth', 'Larksfaith', 'Conehead', 'aretall', 'genneo', 'dannyhungssss', 'Haormusic', 'somemusic', 'wayneband2019', 'danniyeats', 'rene_superawesome', 'lanting', 'HsiaoLin', 'yingdaguagua', 'sugarcatband', 'wendydy95', 'andung1220', 'Five_Flavours_Band', 'haowei_arno', 'BLUERINGLAMB', 's4203871017', 'anthonywen', 'Thehopend', 

In [79]:
fans = [] #粉絲
fans_type = [] #粉絲種類
fans_musician = [] #粉絲中有音樂人的數量
follow = [] #追蹤
follow_type = [] #追蹤種類
follow_musician = [] #追蹤中有音樂人的數量

In [83]:
for i in range(0, len(fans_type)):
    print("type", len(fans_type[i]))
    print("fans_num", fans_num[i])

type 294
fans_num 294
type 84
fans_num 84
type 11330
fans_num 18,974
type 74
fans_num 74
type 78
fans_num 78


In [82]:
for i in range(0, len(fans_link)):
    fanfolwebcrawler(fans_link[i], fans, fans_type, fans_num[i]) #抓取粉絲名稱以及粉絲種類

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until


294


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  if sys.path[0] == '':


84
18,974
74
78
927


KeyboardInterrupt: 

In [78]:
fanfolmusician(fans_type, fans_musician) #計算粉絲中音樂人以及認證音樂人的數量

[]

In [ ]:
for i in range(0, len(follow_link)):
    fanfolwebcrawler(follow_link[i], follow, follow_type, follow_num[i]) #抓取粉絲名稱以及粉絲種類

In [ ]:
fanfolmusician(follow_type, follow_musician)

In [77]:
print(fans)
print(len(fans))
print(fans_type)
print(len(fans_type))
print(fans_musician)
print(len(fans_musician))
print(follow)
print(len(follow))
print(follow_type)
print(len(follow_type))
print(follow_musician)
print(len(follow_musician))

[]
0
[]
0
[]
0
[]
0
[]
0
[]
0


In [ ]:
df1_3 = pd.DataFrame(zip(rb_id, num, fans_num, follow_num, tp, live, fans_musician, follow_musician), 
                   columns = ["音樂人 ID", "音樂數", "粉絲人數", "追蹤中人數", "身分(音樂人/認證音樂人)",
                              "居住地", "粉絲人數中，音樂人的人數", "追蹤對象中，音樂人的人數"])
 
export_csv = df1.to_csv(r'df1_3.csv', header = True, index = False, encoding = "UTF-8")
print(df1_3)

In [27]:
df1_3 = pd.DataFrame(zip(rb_id, num, fans_num, follow_num, tp, live), 
                   columns = ["音樂人 ID", "音樂數", "粉絲人數", "追蹤中人數", "身分(音樂人/認證音樂人)",
                              "居住地"])
 
export_csv = df1_3.to_csv(r'df1_3_2.csv', header = True, index = False, encoding = "UTF-8")
print(df1_3)

                  音樂人 ID 音樂數   粉絲人數 追蹤中人數 身分(音樂人/認證音樂人)  居住地
0      buddhacomfortable  10     23     0           音樂人  臺中市
1              sasa0720h  11  1,649    12         認證音樂人  桃園市
2     ChuNoodleChuNoodle  11    741    46         認證音樂人  新北市
3                  8bkuo   1    156     8           音樂人  新北市
4    anthony0926handsome  21  4,746     3         認證音樂人  臺北市
..                   ...  ..    ...   ...           ...  ...
269             Sophiehu   4    433    17           音樂人  臺北市
270             LexieLiu  14  8,829     0           音樂人   北京
271             danhsueh   8  1,040     9         認證音樂人  臺北市
272    madeinbackstation   5  5,246     6         認證音樂人  桃園市
273         groovy__wave   4     62     0           音樂人   香港

[274 rows x 6 columns]
